In [6]:
import duckdb 
import pandas as pd

In [2]:
conn =duckdb.connect("my_db.db")

In [26]:
#create table for green_tripdata


In [4]:
conn.sql("create table green_tripdata as select * from read_parquet('/workspaces/DataEng/green_tripdata_2025-11.parquet')")

In [7]:
conn.sql("select * from green_tripdata limit 10").df()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1,74,42,1,0.74,7.2,...,0.5,1.94,0.0,NaN,1.0,11.64,1,1,0.00,0.00
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1,74,42,2,0.95,7.2,...,0.5,0.00,0.0,NaN,1.0,9.70,2,1,0.00,0.00
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1,83,160,1,2.19,13.5,...,0.5,5.00,0.0,NaN,1.0,21.00,1,1,0.00,0.00
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1,166,127,1,5.44,24.7,...,0.5,0.50,0.0,NaN,1.0,27.70,1,1,0.00,0.00
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1,166,262,1,3.20,18.4,...,1.5,1.00,0.0,NaN,1.0,24.65,1,1,2.75,0.00
5,1,2025-11-01 00:42:13,2025-11-01 01:04:50,N,1,112,48,2,5.10,26.8,...,1.5,6.55,0.0,NaN,1.0,39.35,1,1,2.75,0.75
6,2,2025-11-01 00:05:41,2025-11-01 00:39:20,N,1,83,87,1,9.80,43.6,...,0.5,9.92,0.0,NaN,1.0,59.52,1,1,2.75,0.75
7,2,2025-11-01 00:42:14,2025-11-01 01:13:20,N,1,66,233,1,5.01,28.9,...,0.5,6.98,0.0,NaN,1.0,41.88,1,1,2.75,0.75
8,2,2025-11-01 00:03:08,2025-11-01 00:06:27,N,1,223,223,1,0.63,5.1,...,0.5,1.52,0.0,NaN,1.0,9.12,1,1,0.00,0.00
9,2,2025-11-01 00:56:33,2025-11-01 01:01:34,N,1,130,130,1,1.15,7.9,...,0.5,0.00,0.0,NaN,1.0,10.40,2,1,0.00,0.00


In [ ]:
#mcq 3

In [17]:
conn.sql("select count(trip_distance) from green_tripdata where (lpep_pickup_datetime::DATE BETWEEN DATE '2025-11-01' AND DATE '2025-11-30')  and trip_distance <=1  ")

┌──────────────────────┐
│ count(trip_distance) │
│        int64         │
├──────────────────────┤
│                 8007 │
└──────────────────────┘

In [ ]:
#mcq 4

In [25]:
conn.sql("""select lpep_pickup_datetime from (select cast(lpep_pickup_datetime as date) lpep_pickup_datetime, max(trip_distance) as max_dist from green_tripdata 
where trip_distance < 100 group by lpep_pickup_datetime order by max_dist desc limit 1 )""")

┌──────────────────────┐
│ lpep_pickup_datetime │
│         date         │
├──────────────────────┤
│ 2025-11-14           │
└──────────────────────┘

In [ ]:
#create table for zone

In [27]:
conn.sql("create table zones as select * from read_csv('/workspaces/DataEng/taxi_zone_lookup.csv')")

In [28]:
conn.sql("select * from zones limit 10").df()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
5,6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone
6,7,Queens,Astoria,Boro Zone
7,8,Queens,Astoria Park,Boro Zone
8,9,Queens,Auburndale,Boro Zone
9,10,Queens,Baisley Park,Boro Zone


In [ ]:
#mcq 5

In [31]:
conn.sql("""select zone from  (select PULocationID,Zone,sum(total_amount) as total
from green_tripdata a 
join zones b on a.PULocationID=b.LocationID	
where cast(lpep_pickup_datetime as date) ='2025-11-18' group by  PULocationID,Zone order by total desc limit 1)""")

┌───────────────────┐
│       Zone        │
│      varchar      │
├───────────────────┤
│ East Harlem North │
└───────────────────┘

In [ ]:
#mcq 6

In [49]:
conn.sql("""select zone from  (select a.PULocationID,du.zone,a.tip_amount 
from green_tripdata a
join zones pu on a.PULocationID=pu.LocationID
join zones du on a.DOLocationID=du.LocationID	
where cast(lpep_pickup_datetime as date)  between '2025-11-01' and '2025-11-30'  
and pu.zone='East Harlem North'
order by a.tip_amount desc limit 1)""")

┌────────────────┐
│      Zone      │
│    varchar     │
├────────────────┤
│ Yorkville West │
└────────────────┘